In [1]:
import numpy as np
import pywt
from scipy.io import loadmat


In [2]:
# Load data
file_path = r"C:\Users\gagan\Downloads\5e128x128LegendreDefault.mat"
data = loadmat(file_path)


In [3]:
# Extract 'abundanciesGT' and 'endmembersGT'
abundancies_gt = data['abundanciesGT']  # Shape: (128, 128, n_abundancies)
endmembers_gt = data['endmembersGT']    # Shape: (n_bands, n_endmembers)


In [4]:
# Reshape 'abundanciesGT' into a 2D array where each pixel's data is a 1D vector
abundancies_vector_data = abundancies_gt.reshape(-1, abundancies_gt.shape[2])  # Shape: (128*128, n_abundancies)


In [5]:
# Reshape 'endmembersGT' as its columns represent 1D vectors
endmembers_vector_data = endmembers_gt.T  # Shape: (n_endmembers, n_bands)


In [6]:
# Perform wavelet transform on the first vectors to get coefficient sizes
sample_abundance_vector = abundancies_vector_data[0]
sample_endmember_vector = endmembers_vector_data[0]

cA_abundance, cD_abundance = pywt.dwt(sample_abundance_vector, 'bior1.3')
cA_endmember, cD_endmember = pywt.dwt(sample_endmember_vector, 'bior1.3')


In [7]:
# Initialize cubes based on coefficient sizes
abundance_approximation_cube = np.empty((abundancies_vector_data.shape[0], cA_abundance.shape[0]))
abundance_detail_cube = np.empty((abundancies_vector_data.shape[0], cD_abundance.shape[0]))

endmember_approximation_cube = np.empty((endmembers_vector_data.shape[0], cA_endmember.shape[0]))
endmember_detail_cube = np.empty((endmembers_vector_data.shape[0], cD_endmember.shape[0]))


In [8]:
# Apply wavelet transform to each vector and store coefficients in cubes
for idx, vector in enumerate(abundancies_vector_data):
    cA, cD = pywt.dwt(vector, 'bior1.3')
    abundance_approximation_cube[idx, :] = cA
    abundance_detail_cube[idx, :] = cD

for idx, vector in enumerate(endmembers_vector_data):
    cA, cD = pywt.dwt(vector, 'bior1.3')
    endmember_approximation_cube[idx, :] = cA
    endmember_detail_cube[idx, :] = cD


In [9]:
# Reshape abundance coefficient cubes back to spatial dimensions
abundance_approximation_cube = abundance_approximation_cube.reshape(abundancies_gt.shape[0], abundancies_gt.shape[1], -1)
abundance_detail_cube = abundance_detail_cube.reshape(abundancies_gt.shape[0], abundancies_gt.shape[1], -1)


In [10]:
# Output shapes for verification
print("Shape of abundanciesGT:", abundancies_gt.shape)
print("Shape of abundance approximation cube:", abundance_approximation_cube.shape)
print("Shape of abundance detail cube:", abundance_detail_cube.shape)


Shape of abundanciesGT: (128, 128, 5)
Shape of abundance approximation cube: (128, 128, 5)
Shape of abundance detail cube: (128, 128, 5)


In [11]:

# Output shapes for endmembers
print("Shape of endmembersGT:", endmembers_gt.shape)
print("Shape of endmember approximation cube:", endmember_approximation_cube.shape)
print("Shape of endmember detail cube:", endmember_detail_cube.shape)


Shape of endmembersGT: (5, 431)
Shape of endmember approximation cube: (431, 5)
Shape of endmember detail cube: (431, 5)
